In [1]:
!pip install transformers accelerate ipywidgets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.3 MB/s eta 0:00:00


In [2]:
import ipywidgets as widgets
from IPython.display import display, Markdown
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch


In [3]:
# Load the open model (TinyLlama is open, small, and quick for demo)
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="cpu")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Widgets for interactive input
role_input = widgets.Text(
    value='Data Scientist',
    placeholder='Enter job role or paste resume',
    description='Role/Resume:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

num_questions_input = widgets.BoundedIntText(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Questions:',
    disabled=False
)

generate_button = widgets.Button(description='Generate', button_style='primary')
output = widgets.Output()

def build_questions_prompt(role_or_resume, num_questions):
    return (
        f"You are an expert interviewer. Generate {num_questions} technical and relevant interview questions "
        f"for the following job role or resume. Only return the questions, numbered.\n"
        f"Job Role/Resume: {role_or_resume}\nQuestions:"
    )

def build_answer_prompt(question, role_or_resume):
    return (
        f"For the job role or resume below, provide a strong, concise sample answer to the interview question.\n"
        f"Job Role/Resume: {role_or_resume}\n"
        f"Question: {question}\n"
        "Sample Answer:"
    )

def get_questions(role_or_resume, num_questions):
    prompt = build_questions_prompt(role_or_resume, num_questions)
    response = generator(
        prompt,
        max_length=256,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True  # <- Fix: remove warning
    )
    import re
    lines = response[0]['generated_text'].split('\n')
    questions = []
    for line in lines:
        match = re.match(r'^\s*\d+[).\-]?\s*(.*)', line)
        if match:
            questions.append(match.group(1).strip())
        elif line.strip() and len(questions) < num_questions:
            questions.append(line.strip())
        if len(questions) >= num_questions:
            break
    # fallback if no numbered list is detected
    if not questions:
        # Try to split by common delimiters and filter short lines
        for line in lines:
            if len(line.strip()) > 10:
                questions.append(line.strip())
            if len(questions) >= num_questions:
                break
    return questions[:num_questions]

def get_answer(question, role_or_resume):
    prompt = build_answer_prompt(question, role_or_resume)
    response = generator(
        prompt,
        max_length=128,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True  # <- Fix: remove warning
    )
    answer = response[0]['generated_text'].split("Sample Answer:")[-1].strip()
    # Remove repeated question in answer
    if answer.lower().startswith(question.lower()):
        answer = answer[len(question):].strip()
    return answer

def on_submit(sender):
    with output:
        output.clear_output()
        role = role_input.value.strip()
        n = num_questions_input.value
        if not role:
            display(Markdown("**Please enter a job role or paste a resume.**"))
            return
        display(Markdown(f"**Generating {n} interview questions for:** `{role}`"))
        questions = get_questions(role, n)
        for idx, q in enumerate(questions, 1):
            display(Markdown(f"**Q{idx}. {q}**"))
            display(Markdown("_Generating sample answer..._"))
            answer = get_answer(q, role)
            display(Markdown(f"**Sample Answer:** {answer}"))
            display(Markdown("---"))

generate_button.on_click(on_submit)

display(role_input, num_questions_input, generate_button, output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Device set to use cpu


Text(value='Data Scientist', description='Role/Resume:', layout=Layout(width='50%'), placeholder='Enter job ro…

BoundedIntText(value=5, description='Questions:', max=10, min=1)

Button(button_style='primary', description='Generate', style=ButtonStyle())

Output()